In [ ]:
pip install pymongo json

In [ ]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")

mydb = myclient["dbtest"]
mycol = mydb["CPE"]

In [ ]:
import json
with open('data.json', 'r') as file:
    data = json.load(file)

In [ ]:
data = data['cpe-list']['cpe-item']
len(data)

In [ ]:
from datetime import datetime

extracted = []

for item in data:
    title = item.get('title', {})
    if isinstance(title, list):
        title = title[0].get('#text')
    else:
        title = title.get('#text')
    refs = item.get('references', {}).get('reference', [])
    if isinstance(refs, dict):
        refs = [refs]
    references = [ref.get('@href') for ref in refs if '@href' in ref]
    deprecation_date = item.get('@deprecation_date')
    if not deprecation_date:
        deprecation_date = item.get('cpe-23:cpe23-item', {}).get('cpe-23:deprecation', {}).get('@date')
    if deprecation_date:
        deprecation_date = deprecation_date[:10]
        try:
            deprecation_date = datetime.strptime(deprecation_date, "%Y-%m-%d")
        except Exception:
            deprecation_date = None
    else:
        deprecation_date = None
    cpe23_url = item.get('cpe-23:cpe23-item', {}).get('@name')
    extracted.append({
        "title": title,
        "cpe23_url": cpe23_url,
        "references": references,
        "deprecation_date": deprecation_date
    })


In [ ]:
print(extracted[1])
len(extracted)

In [ ]:
mycol.insert_many(extracted)